In [170]:
import pandas as pd
import numpy as np

In [171]:
# Read the inflation.csv file into a dataframe
fred_df = pd.read_csv('FRED_merged.csv')

# Read the population.csv file into a dataframe
downloaded_df = pd.read_csv('downloaded_merged.csv')

In [185]:
downloaded_df.head()

,country,date,bot,inflation,interest,liquidity,debt
0,Afghanistan,2017,"-6,803.56",NaN,NaN,NaN,NaN
1,Afghanistan,2018,"-6,378.17",NaN,NaN,NaN,NaN
2,Afghanistan,2019,"-5,855.06",149.9,...,"7,426.98",NaN
3,Afghanistan,2020,"-5,506.55",...,...,"8,419.49",NaN
4,Afghanistan,2021,...,...,...,...,NaN


In [173]:
downloaded_df['country']=downloaded_df.country.str.split(',').str[0]

In [174]:
fred_df['date']=pd.to_datetime(fred_df['date']).dt.year

In [175]:
fred_df['date'].dtypes,downloaded_df.date.dtypes

(dtype('int32'), dtype('int64'))

In [176]:
fred_df[['country', 'date']].dtypes

country    object
date        int32
dtype: object

In [186]:
merged_df = pd.merge(fred_df, downloaded_df, on=['country', 'date'], how='outer')
merged_df.head()

,country,date,Fiscal Balance,GDP per capita,GDP,Unemployment Rate,Inflation,Population,CPI,bot,inflation,interest,liquidity,debt
0,Aruba,2018,NaN,31705.279053,3.202235e+09,NaN,3.62604141352956,105962,3.626041,287.4,NaN,NaN,NaN,NaN
1,Aruba,2018,NaN,31705.279053,3.202235e+09,NaN,3.62604141352956,105962,3.626041,NaN,105.06,6.00,852.08,NaN
2,Aruba,2019,NaN,31762.734026,3.368970e+09,NaN,4.25746204335428,106442,4.257462,321.66,109.53,5.70,830.00,NaN
3,Afghanistan,2018,1.63419574,502.056771,1.841885e+10,17.466,0.62614913,.,NaN,"-6,378.17",NaN,NaN,NaN,NaN
4,Afghanistan,2018,1.63419574,502.056771,1.841885e+10,17.466,0.62614913,.,NaN,NaN,146.52,...,"7,305.68",NaN


In [187]:
clean_merged_df = merged_df.groupby(['country', 'date']).agg({'Unemployment Rate': 'sum', 'Population': 'sum','Fiscal Balance':'sum',
                                                            'GDP per capita':'sum','GDP':'sum','Inflation':'sum',
                                                            'bot':'sum','interest':'sum','liquidity':'sum','debt':'sum',
                                                             'CPI':'sum','inflation':'sum'}).reset_index()

pd.set_option('display.max_rows', None)
clean_merged_df.head()

,country,date,Unemployment Rate,Population,Fiscal Balance,GDP per capita,GDP,Inflation,bot,interest,liquidity,debt,CPI,inflation
0,Advanced Economies,2017,0.0,0,0,0.0,0.0,0,0,0,"4,822,093.35",0,0.0,0
1,Advanced Economies,2018,0.0,0,0,0.0,0.0,0,0,0,"4,852,344.48",0,0.0,0
2,Advanced Economies,2019,0.0,0,0,0.0,0.0,0,0,0,"5,018,887.66",0,0.0,0
3,Advanced Economies,2020,0.0,0,0,0.0,0.0,0,0,0,"5,635,428.32",0,0.0,0
4,Advanced Economies,2021,0.0,0,0,0.0,0.0,0,0,0,"6,148,274.59",0,0.0,0


In [188]:
df1 = clean_merged_df.loc[:, ['country', 'date','GDP']]
df1.head()

,country,date,GDP
0,Advanced Economies,2017,0.0
1,Advanced Economies,2018,0.0
2,Advanced Economies,2019,0.0
3,Advanced Economies,2020,0.0
4,Advanced Economies,2021,0.0


In [189]:

df_clean= df1.replace(0, np.nan)  # Replace zero values with NaN
df_clean = df_clean.dropna() # Remove rows containing NaN values

df_clean.head()




,country,date,GDP
6,Afghanistan,2018,5.525654e+10
7,Afghanistan,2019,3.780898e+10
8,Afghanistan,2020,4.028688e+10
9,Afghanistan,2021,2.957372e+10
11,Albania,2018,8.120940e+05


In [190]:
count_values = df_clean.groupby('country').size().reset_index(name='Count')

count_values.head()

,country,Count
0,Afghanistan,4
1,Albania,4
2,Algeria,4
3,American Samoa,4
4,Angola,4


In [191]:

gdp_growth = (df_clean.groupby('country')['GDP']
                .apply(lambda x: (((x.iloc[-1])-(x.iloc[0]))/x.iloc[0])*100)
                .reset_index(name='GDP growth'))

gdp_growth.head()



,country,GDP growth
0,Afghanistan,-46.479239
1,Albania,-45.082367
2,Algeria,-58.316796
3,American Samoa,10.322663
4,Angola,-57.051030


In [192]:
merged_df = pd.merge(gdp_growth, count_values, on='country', how='inner')

merged_df.head()

,country,GDP growth,Count
0,Afghanistan,-46.479239,4
1,Albania,-45.082367,4
2,Algeria,-58.316796,4
3,American Samoa,10.322663,4
4,Angola,-57.051030,4


In [193]:
merged_df['Agg. GDP Growth'] = merged_df['GDP growth'] / merged_df['Count']


merged_df.head()

,country,GDP growth,Count,Agg. GDP Growth
0,Afghanistan,-46.479239,4,-11.619810
1,Albania,-45.082367,4,-11.270592
2,Algeria,-58.316796,4,-14.579199
3,American Samoa,10.322663,4,2.580666
4,Angola,-57.051030,4,-14.262758
